# Movie Ratings Dataset with Spark

We'll be building a recommender system using Spark. This notebook will start by doing basic things like counting the number of ratings for each rating category (1 star, 2 stars, 3 stars, etc.) to something more complex like a recommender system.

The dataset we are working with can be found here: https://grouplens.org/datasets/movielens/

We'll start by using the MovieLens 100K dataset which contains 100,000 ratings from 1000 users on 1700 movies.

## Part 1: Counting the number of ratings for star category

In [1]:
import collections

lines = sc.textFile("/Users/jacquesthibodeau/big-data-datasets/ml-100k/u.data")
ratings = lines.map(lambda x: x.split()[2])
result = ratings.countByValue()

sortedResults = collections.OrderedDict(sorted(result.items()))
for key, value in sortedResults.items():
    print("%s %i" % (key, value))

1 6110
2 11370
3 27145
4 34174
5 21201


# Part 2: What are the most popular movies?

We're going to figure out which movies are the most popular.

In [2]:
# first, let's create a function to load the movie names

def loadMovieNames():
    movieNames = {}
    with open("/Users/jacquesthibodeau/big-data-datasets/ml-100k/u.item") as f:
        for line in f:
            fields = line.split('|')
            movieNames[int(fields[0])] = fields[1]
        return movieNames
    
nameDict = sc.broadcast(loadMovieNames())

movies = lines.map(lambda x: (int(x.split()[1]), 1))
movieCounts = movies.reduceByKey(lambda x, y: x + y)

flipped = movieCounts.map(lambda x : (x[1], x[0]))
sortedMovies = flipped.sortByKey()

sortedMoviesWithNames = sortedMovies.map(lambda countMovie : (nameDict.value[countMovie[1]], countMovie[0]))

results = sortedMoviesWithNames.collect()

for result in results:
    print(result)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 2892: invalid continuation byte